In [ ]:
import juliacall as jl

In [ ]:
jl.Main.include("julia_ref/code_00_Lattice.jl")
jl.Main.include("julia_ref/code_01_FiniteLattice.jl")
jl.Main.include("julia_ref/code_10_InteractionMatrix.jl")
jl.Main.include("julia_ref/code_20_ManyBodyHamiltonian.jl")

In [ ]:
pkgs = "CSV DataFrames ProgressMeter SuiteSparseGraphBLAS KrylovKit Combinatorics Colors JLD2 LaTeXStrings".split()

for pkg in pkgs:
    jl.Main.Pkg.add(pkg)